In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="1"
import tensorflow as tf
from keras.utils import image_dataset_from_directory


train_dir = "./data/cats_dogs/train"
test_dir = "./data/cats_dogs/test"

train_generator, validation_generator = image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="both",
  seed=42,
  image_size=(150, 150),
  batch_size=20
  )

test_generator = image_dataset_from_directory(
  test_dir,
  image_size=(150, 150),
  batch_size=20
  )

In [ ]:
from keras import optimizers
from keras import models, layers


model = models.Sequential()

# model.add(layers.RandomBrightness(factor=[-0.2, 0.2]))
# model.add(layers.RandomContrast(factor=0.2))
# model.add(layers.RandomFlip())
# model.add(layers.RandomRotation(factor=[-0.2, 0.2]))
# model.add(layers.RandomTranslation(height_factor=0.2, width_factor=0.2))
# model.add(layers.RandomZoom(height_factor=[-0.2, 0.2]))

model.add(layers.Rescaling(scale=1./255))

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
history = model.fit(
      x=train_generator,
      epochs=30,
      steps_per_epoch=800,
      validation_data=validation_generator,
      validation_steps=20,
      verbose=2
)

In [ ]:
import matplotlib.pyplot as plt


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure()
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.evaluate(test_generator)